# Connecting to a Live SQL Server Database
## Introduction
There are several ways that we can use Python to create a live connection to a database. In this example, we will be using a package called "SQL Alchemy" to connect to the SQL Server instance that you created on your computer.

Pulling data from a live database connection is essential in some applications. For example, when working with stock market data to determine whether or not to sell or buy a certain stock, you might need the most recent data possible in every moment. Live data might also be important in manufacturing, where repeated signals sent to the database can be an indication of imminent machine failure or the need to inspect a certain aspect of production. A live data connection can also be useful in instances that are not time-critical, such as looking at Human Resources data, when asking for a CSV of data from the HR manager each time you want to run an analysis may be repetitive and unneccesary. Finally, live connections ensure that data is not actually stored on the computer using the data, ensuring that confidential data is not lost if the computer is lost.

Using a live data connection, however, comes at a cost. Because your computer has to wait for the data to come from the database, the actual code that runs the analysis may take longer to process. The analyses also become more difficult to reproduce, since the data frequently changes. Furthermore, there may be a substantial monetary price to pay for maintaining live connections with data from your computer.

The most difficult aspect of using a live data connection, however, probably has to do with the actual infrastructure of the database in the organization. Before anything else, the organization has to have a server set up somewhere that is open to accepting connections from other computers. Then, somebody has to be in charge of managing all of the connections to the servers. Permissions become a hassle to manage, since each computer may require different levels of access to different parts of the database. In very large organizations, there may even be several databases managed by different departments with varying methods of access. Some departments may feel confident in how they manage permissions in their database, and others may not trust people outside of their department to access their data with a live connection. 

For these reasons, data is often provided to data practitioners in a CSV or similar file. However, live connections are preferred in many cases. In this notebook, we will connect to the live SQL Server instance that you created on your computer during the course *SQL Fundamentals*.

## Installing SQL Server
In the off-chance that you don't have SQL Server installed on your computer yet, you should do so before continuing on and running the rest of the code in this file. The code will not run -- you cannot connect to a database that you don't have installed.

You can use the following Canvas pages to install SQL Server and the required databases on your computer:

1. [Installing SQL Server 2019](https://btech.instructure.com/courses/540621/pages/installing-sql-server-2019)
2. [Installing Microsoft SQL Management Studio](https://btech.instructure.com/courses/540621/pages/installing-microsoft-sql-management-studio)
3. [Connection to the SQL Server](https://btech.instructure.com/courses/540621/pages/connecting-to-the-sql-server)
4. [Loading in a Database](https://btech.instructure.com/courses/540621/pages/loading-in-a-database)

## Installing the required libraries
You can install the required libraries, `pyodbc` and `sqlalchemy`, by running the following command in a Command Prompt window.

```
pip install pyodbc sqlalchemy
```
I have had issues installing `pyodbc` in the past. If you find that you are getting download errors when trying to download either of these libraries, reach out to me and I will help you troubleshoot installation. Anything that has to do with connecting two computers together is usually fairly complex, including the installation of the libraries, so don't get worried if you do get a few errors during this process.

## Connecting to SQL Server
### Import the libraries
As with every Python project, before doing anything else, we should import the libraries that we need for our analysis. In this case, we will need to import `sqlalchemy` and `pandas`. Note that the Pandas library is almost *always* imported under the alias `pd`, but sqlalchemy doesn't really have a naming convention yet. In this example, I'll import it under the alias `sql`.

Also note that in the code below, we never actually import the `pyodbc` library into our code. This library needs to be installed on the computer so that SQL Alchemy can make the database connection to a SQL Server instance, but doesn't actually need to be imported to the code.

In [ ]:
import sqlalchemy as sql
import pandas as pd

### Creating the connection information
Before connecting to the live database, we need to create some variables that contain information about the connection that our computer is going to make. 

First, we define the `driver`, which tells SQL Alchemy which driver to use to connect to the database. There are different drivers for different kinds of databases, but the driver "ODBC Driver 17 for SQL Server" basically says to use the Open Database Connectivity protocol specifically configured for SQL Server to make the database connection. You probably don't need to change this.

Next, we define the `server`. If we remember back to the SQL course, at some times we connected Microsoft SQL Server Management Studio to the `BDA` server located at Bridgerland Technical College. If we wanted to connect to that server, we would set `server` equal to `BDA`. However, because I am just going to access the SQL Server instance on my own computer, the `server` is going to be the name of my computer followed by a backslash `\` and then the instance name.

You can find out what `server` should be by looking at SQL Server Management Studio and using the drop down menu next to "Server Name" to look at past database connections. You will probably see something similar to `LT-W1828-HH\SQLSERVER2019`, where everything to the left of the `\` is the server (computer) name and everything to the right of the `\` is the SQL Server instance name. **Copy the entire string (the server name and instance name) and paste it in to create the variable `server`**.

<img src="./img/2022-05-24_14-23-30.png" style="margin:auto;margin-bottom:1rem">

Finally, we define the database that we are going to access in the variable `db`. In this case, I am going to access the database "AdventureWorks2019".

Note that you will have to change the `server` variable below to match what your server is. You may also need to change the `db` variable if you are connecting to a database that is not AdventureWorks2019.

In [ ]:
driver = "ODBC Driver 17 for SQL Server"
server = "LT-W1828-HH\SQLSERVER2019" # Change me
db = "AdventureWorks2019"

There are other variables we could define as well. For example, it might actually be a good idea to create a new database user for your code that has different permissions from you personally. Your code probably doesn't need access to all of the databases that you do, in other words. To accomplish this task, you could create other parameters `username` and `password` that tell SQL Alchemy how to log in to the database as a different user.

In this case, we won't pass in any username or password at all. This allows SQL Alchemy to log in to the database using Windows Authentication, which does not require a username and password.

### Creating the engine

Now that we have our connection information defined, we can pass these variables into a string, which we can then pass into the SQL Alchemy `create_engine()` function. This function returns an `Engine` object, which is basically just a string that knows how to interpret itself so that it can be used by SQL Server specifically. You don't have to know exactly how this works.

All we need to do to create the engine is pass in a formatted string into the `create_engine()` function.

In [ ]:
connection_string = f"mssql+pyodbc://{server}/{db}?driver={driver}"
engine = sql.create_engine(connection_string)

The formatted string `connection_string` looks complicated, but really, it's just a long string of text that will tell SQL Alchemy how to connect to the database. The first part, `mssql` says that the target database is Microsoft SQL Server. Then, `pyodbc` says to use the `pyodbc` library (which we installed previously) to interpret the commands to the database. `server` and `db` take the variables created above and explain where the database is located, and `driver={driver}` takes the variable from above and tells SQL Alchemy which driver of the OBDC protocol to use.

We then pass the `connection_string` into the `create_engine()` function to create an `Engine` object.

If none of that made sense, don't worry. Most people look these things up when they need them, so all you need is to be aware that database connections are created as strings.

### Writing a SQL query

Ahh! Finally something that looks familiar.

SQL Alchemy provides ways to retrieve data from the database that don't require knowledge of SQL. However, since we are already familiar with SQL, it will probably be just as easy to write raw SQL and save it to a variable.

In the code below, I use triple quotes to save a SQL query to a variable `query` as a string. Triple quotes make it easier to write SQL queries that span multiple lines.

In [ ]:
query = '''
    SELECT *
    FROM HumanResources.Department
    ;
'''

### Getting the data out
Again, there are many ways to get data out of a database using SQL Alchemy. However, in my opinion, one of the easiest ways is using the Pandas function `read_sql()`. This function is similar to the `read_csv()` function but takes two parameters: a SQL query (a string) and a database connection.

First, we use the SQL Alchemy engine from above to create a database connection called `connection`. We can do this by using the `.connect()` method on the `engine` variable that we created previously.

In [ ]:
connection = engine.connect()

Finally, the moment of truth. We can pass in the variables `query` and `connection` to the Pandas function `read_sql()` to return a dataframe that contains all of the data from our SQL query.

In [ ]:
pd.read_sql(query, connection)

,DepartmentID,Name,GroupName,ModifiedDate
0,1,Engineering,Research and Development,2008-04-30
1,2,Tool Design,Research and Development,2008-04-30
2,3,Sales,Sales and Marketing,2008-04-30
3,4,Marketing,Sales and Marketing,2008-04-30
4,5,Purchasing,Inventory Management,2008-04-30
5,6,Research and Development,Research and Development,2008-04-30
6,7,Production,Manufacturing,2008-04-30
7,8,Production Control,Manufacturing,2008-04-30
8,9,Human Resources,Executive General and Administration,2008-04-30
9,10,Finance,Executive General and Administration,2008-04-30


Voilà! We now have a Pandas dataframe. We can now save the results of the `read_sql()` function above to a variable and interact with the dataframe just like we would with any other dataframe.

In [ ]:
df = pd.read_sql(query, connection)

In [ ]:
df.head()

,DepartmentID,Name,GroupName,ModifiedDate
0,1,Engineering,Research and Development,2008-04-30
1,2,Tool Design,Research and Development,2008-04-30
2,3,Sales,Sales and Marketing,2008-04-30
3,4,Marketing,Sales and Marketing,2008-04-30
4,5,Purchasing,Inventory Management,2008-04-30
